In [2]:
import pandas as pd
import re
import os
from dotenv import load_dotenv
from github import Github
from tqdm import tqdm
from datetime import datetime
import pytz
import seaborn as sns
import matplotlib.pyplot as plt



In [3]:
org_name = "Rdatatable"
repo_name = "data.table"

reponame_noperiod = repo_name.replace(".", "")
reponame_noperiod = reponame_noperiod.replace("_", "")
reponame_noperiod = reponame_noperiod.lower()

In [ ]:
g = Github(secret)
org = g.get_organization(org_name)
repo = org.get_repo(repo_name)
g.rate_limiting

(4998, 5000)

In [5]:
file_path_comments = f'Files/{org_name.lower()}_{reponame_noperiod}_pull_request_comments.xlsx'

file_path = f'Files/{org_name.lower()}_{reponame_noperiod}_pull_requests.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    pull_df = pd.read_excel(file_path, engine='openpyxl')
    print("Pull Request file exist")
else:
    pull_df = pd.DataFrame()
    print("Error:Pull Request file does not exist ", file_path)

if os.path.exists(file_path_comments):
    pull_df_comments = pd.read_excel(file_path_comments, engine='openpyxl')
    print("Pull Request Comments file exist")
else:
    pull_df_comments = pd.DataFrame()
    print("Error:Pull Request Comments file does not exist ",file_path_comments)

# Define the file path
file_path_comments = f'Files/{org_name.lower()}_{reponame_noperiod}_issues_comments.xlsx'

file_path = f'Files/{org_name.lower()}_{reponame_noperiod}_issues.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    issues_df = pd.read_excel(file_path, engine='openpyxl')
    print("Issues Request file exists")
else:
    issues_df = pd.DataFrame()
    print("Error: Issues Request file does not exist ", file_path )
if os.path.exists(file_path_comments):
    issues_df_comments = pd.read_excel(file_path_comments, engine='openpyxl')
    print("Issues Request Comments file exists")
else:
    issues_df_comments = pd.DataFrame()
    print("Error: Issues Comments file does not exist ", file_path_comments)

Pull Request file exist
Pull Request Comments file exist
Issues Request file exists
Issues Request Comments file exists


In [6]:

def standardize_timestamps(df, column_name='created_at', new_column_name='time'):
    """
    Convert timestamps in a DataFrame column to London time (GMT/UTC+0).
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the timestamp column
    column_name : str
        Name of the column containing timestamps with timezone info
    new_column_name : str
        Name of the new column to store London time
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with the new London time column added
    """
    # Make sure timestamps are parsed as datetime with timezone info
    df[column_name] = pd.to_datetime(df[column_name], utc=True)
    
    # Convert to London time
    london_tz = pytz.timezone('Europe/London')
    df[new_column_name] = df[column_name].dt.tz_convert(london_tz)
    
    # Create a datetime column without timezone info for plotting
    df[f'{new_column_name}_naive'] = df[new_column_name].dt.tz_localize(None)
    
    return df

pull_df=standardize_timestamps(pull_df)
pull_df_comments=standardize_timestamps(pull_df_comments)
issues_df=standardize_timestamps(issues_df)
issues_df_comments=standardize_timestamps(issues_df_comments)

#get rid of created_at columns
pull_df.drop(columns=['created_at'], inplace=True)
pull_df_comments.drop(columns=['created_at'], inplace=True)
issues_df.drop(columns=['created_at'], inplace=True)
issues_df_comments.drop(columns=['created_at'], inplace=True)

 



In [7]:
pull_df['created_by'] = pull_df['created_by'].apply(lambda x: x.split('/')[-1])
issues_df['created_by'] = issues_df['created_by'].apply(lambda x: x.split('/')[-1])
pull_df_comments['created_by'] = pull_df_comments['created_by'].apply(lambda x: re.search(r'login="([^"]+)"', x).group(1))
issues_df_comments['created_by'] = issues_df_comments['created_by'].apply(lambda x: re.search(r'login="([^"]+)"', x).group(1))

## Fliter out users that have only contributed once across files.

In [8]:
# make copies of all the dataframes
pull_df_all = pull_df.copy()
pull_df_comments_all = pull_df_comments.copy()
issues_df_all = issues_df.copy()
issues_df_comments_all = issues_df_comments.copy()
#drop "Unnamed: 0", issue_id, id, body from the comments dataframes
issues_df_comments_all.drop(columns=['Unnamed: 0', 'issue_id', 'id', 'body'], inplace=True)
pull_df_comments_all.drop(columns=['Unnamed: 0', 'pull_request_id', 'id', 'body'], inplace=True)
issues_df_all.drop(columns=['Unnamed: 0', 'id', 'title','closed_at','labels','closed_by'], inplace=True)
pull_df_all.drop(columns=['Unnamed: 0', 'id', 'title','closed_at','labels','state','number_commits','number_files_modified','mergeable_status'], inplace=True)
#add all the dataframes together making one long file. add one more colum to say if the row was added from the issues or pull requests
pull_df_all['type'] = 'pull'
issues_df_all['type'] = 'issue'
pull_df_comments_all['type'] = 'pull'
issues_df_comments_all['type'] = 'issue'

#add all the dataframes together making one long file. add one more colum to say if the row was added from the issues or pull requests
all_df = pd.concat([pull_df_all, issues_df_all,pull_df_comments_all,issues_df_comments_all])

def calculate_user_metrics_all(df, user_column='created_by', type_column='type'):
    user_metrics = df.groupby([user_column]).size().reset_index(name='total_activity')
    user_metrics['active_days'] = df.groupby([user_column])['time_naive'].nunique().values
    
    return user_metrics

# Calculate user metrics for all_df
all_user_metrics = calculate_user_metrics_all(all_df, user_column='created_by', type_column='type')

# if a user has only 1 active day and less than 5 total activitys drop them.
all_user_metrics = all_user_metrics[(all_user_metrics['active_days'] > 1) | (all_user_metrics['total_activity'] > 5)]


# functions to make new data frames

In [9]:
def aggregate_daily_activity(df, user_column='created_by', timestamp_column='time_naive'):
    """
    Aggregate timestamp data into daily activity counts per user.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the user and timestamp columns
    user_column : str
        Name of the column containing user identifiers
    timestamp_column : str
        Name of the column containing timestamps
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with daily activity counts per user
    """
    # Ensure the timestamp column is in datetime format
    df[timestamp_column] = pd.to_datetime(df[timestamp_column], utc=True)
    
    # Extract date from timestamp
    df['date'] = df[timestamp_column].dt.date
    
    # Aggregate daily activity counts per user
    daily_activity = df.groupby([user_column, 'date']).size().reset_index(name='activity_count')
    
    return daily_activity

In [10]:
def calculate_user_metrics(daily_activity, user_column='created_by'):
    """
    Calculate useful metrics about each user's participation patterns.
    
    Parameters:
    -----------
    daily_activity : pandas.DataFrame
        DataFrame with daily activity counts per user
    user_column : str
        Name of the column containing user identifiers
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with user metrics
    """
    user_metrics = daily_activity.groupby(user_column)['activity_count'].agg(
        total_activity='sum',
        mean_activity='mean',
        std_activity='std',
        min_activity='min',
        max_activity='max',
        active_days='count' ).reset_index()
    
    return user_metrics

In [11]:
def create_user_activity_matrix(daily_activity, user_column='created_by', date_column='date'):
    """
    Create a matrix suitable for machine learning.
    
    Parameters:
    -----------
    daily_activity : pandas.DataFrame
        DataFrame with daily activity counts per user
    user_column : str
        Name of the column containing user identifiers
    date_column : str
        Name of the column containing dates
        
    Returns:
    --------
    pandas.DataFrame
        User activity matrix
    """
    user_activity_matrix = daily_activity.pivot(index=user_column, columns=date_column, values='activity_count').fillna(0)
    
    return user_activity_matrix

In [12]:
# Aggregate daily activity for pull requests, pull request comments, issues, and issue comments
pull_daily_activity = aggregate_daily_activity(pull_df, user_column='created_by', timestamp_column='time_naive')
pull_comments_daily_activity = aggregate_daily_activity(pull_df_comments, user_column='created_by', timestamp_column='time_naive')
issues_daily_activity = aggregate_daily_activity(issues_df, user_column='created_by', timestamp_column='time_naive')
issues_comments_daily_activity = aggregate_daily_activity(issues_df_comments, user_column='created_by', timestamp_column='time_naive')


# Calculate user metrics for each type of activity
pull_user_metrics = calculate_user_metrics(pull_daily_activity, user_column='created_by')
pull_comments_user_metrics = calculate_user_metrics(pull_comments_daily_activity, user_column='created_by')
issues_user_metrics = calculate_user_metrics(issues_daily_activity, user_column='created_by')
issues_comments_user_metrics = calculate_user_metrics(issues_comments_daily_activity, user_column='created_by')

# Create user activity matrices for each type of activity
pull_user_activity_matrix = create_user_activity_matrix(pull_daily_activity, user_column='created_by', date_column='date')
pull_comments_user_activity_matrix = create_user_activity_matrix(pull_comments_daily_activity, user_column='created_by', date_column='date')
issues_user_activity_matrix = create_user_activity_matrix(issues_daily_activity, user_column='created_by', date_column='date')
issues_comments_user_activity_matrix = create_user_activity_matrix(issues_comments_daily_activity, user_column='created_by', date_column='date')

In [13]:
merged_df = (
    pull_daily_activity
    .merge(pull_comments_daily_activity, on=['date', 'created_by'], how='outer', suffixes=('_pulls', '_pull_comments'))
    .merge(issues_daily_activity, on=['date', 'created_by'], how='outer')
    .merge(issues_comments_daily_activity, on=['date', 'created_by'], how='outer', suffixes=('_issues', '_issues_comments'))
)

# Fill missing values with 0 since some users may not have activity in all categories
merged_df = merged_df.fillna(0)

# Ensure all activity columns are integers
activity_columns = [col for col in merged_df.columns if col.startswith('activity_count')]
merged_df[activity_columns] = merged_df[activity_columns].astype(int)
# Get the list of users in all_user_metrics
valid_users = all_user_metrics['created_by'].tolist()

# Filter merged_df to keep only rows with users in valid_users
merged_df = merged_df[merged_df['created_by'].isin(valid_users)]

#deleat row 1
merged_df = merged_df.drop(merged_df.index[0])


In [18]:
merged_df

,created_by,date,activity_count_pulls,activity_count_pull_comments,activity_count_issues,activity_count_issues_comments
1,arunsrinivasan,2014-06-09,0,0,0,2
2,fedyakov,2014-06-09,0,0,1,1
3,geneorama,2014-06-09,0,0,1,0
4,juancentro,2014-06-09,0,0,1,0
5,arunsrinivasan,2014-06-10,0,0,0,2
...,...,...,...,...,...,...
12650,codecov[bot],2025-02-07,0,0,0,2
12651,dkutner,2025-02-07,0,0,0,4
12652,github-actions[bot],2025-02-07,0,0,0,1
12653,iagogv3,2025-02-07,0,0,2,1


i wanna try to have the users old data to be used as their trai data and their test be the r data table we have 